In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import GetOldTweets3 as got
import datetime
import json

%matplotlib inline

In [15]:
def get_tweets(username, since_date='2018-01-01', until_date='now'):

    if until_date == 'now':
        until_date = datetime.datetime.now().strftime('%Y-%m-%d')

    # Creation of query object
    tweetCriteria = got.manager.TweetCriteria().setUsername(username)\
        .setSince(since_date).setUntil(until_date).setTopTweets(False)

    # Creation of list that contains all tweets
    tweets = got.manager.TweetManager.getTweets(tweetCriteria)

    return tweets

In [16]:
tweets = get_tweets('strawlab')